## Sudoku Solver using OpenCV

In [9]:
import cv2
import numpy as np

In [10]:
# Load the sudoku puzzle images
puzzle1 = cv2.imread('Puzzles/puzzle1.png')
puzzle2 = cv2.imread('Puzzles/puzzle2.png')

# Display the images
cv2.imshow('Puzzle 1', puzzle1)
cv2.imshow('Puzzle 2', puzzle2)
cv2.waitKey(5)
cv2.destroyAllWindows()

## Process the images

In [20]:
def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (7, 7), 3)
    
    # Apply adaptive thresholding to get binary image
    # Changed THRESH_BINARY_INV to THRESH_BINARY to invert the colors
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY, 11, 2)
    
    return thresh

def find_puzzle(image):
    # Preprocess the image
    processed = preprocess_image(image)
    
    # Find contours - invert image temporarily for contour detection
    processed_inv = cv2.bitwise_not(processed)
    contours, _ = cv2.findContours(processed_inv.copy(), cv2.RETR_EXTERNAL,
                                  cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours by area in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    puzzle_contour = None
    
    # Loop through contours to find the puzzle grid
    for contour in contours:
        # Approximate the contour
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        
        # If we have found a rectangle with 4 corners, we've found our puzzle
        if len(approx) == 4:
            puzzle_contour = approx
            break
            
    return puzzle_contour

# Process both puzzles
puzzle1_processed = preprocess_image(puzzle1)
puzzle2_processed = preprocess_image(puzzle2)

# Find puzzle contours
puzzle1_contour = find_puzzle(puzzle1)
puzzle2_contour = find_puzzle(puzzle2)

# Draw the contours on the original images
if puzzle1_contour is not None:
    cv2.drawContours(puzzle1, [puzzle1_contour], -1, (0, 255, 0), 2)
if puzzle2_contour is not None:
    cv2.drawContours(puzzle2, [puzzle2_contour], -1, (0, 255, 0), 2)

# Display results
cv2.imshow('Processed Puzzle 1', puzzle1_processed)
cv2.imshow('Processed Puzzle 2', puzzle2_processed)
cv2.imshow('Detected Grid 1', puzzle1)
cv2.imshow('Detected Grid 2', puzzle2)
cv2.waitKey(5)
cv2.destroyAllWindows()


## Download digit templates -> Warning: I do not recommend this method. Use the images in the folder instead.

In [5]:
assert False, "I do not recommend this method. Use the images in the folder instead."
import os
import numpy as np
from tensorflow.keras.datasets import mnist
from PIL import Image

# Load the MNIST dataset
(train_images, train_labels), (_, _) = mnist.load_data()

# Specify the output directory
output_dir = 'mnist_digits'
os.makedirs(output_dir, exist_ok=True)

# Define the digits to save
digits_to_save = range(1, 10)

# Initialize a dictionary to track saved digits
saved_digits = {digit: False for digit in digits_to_save}

# Iterate over the dataset to find and save one image per digit
for image, label in zip(train_images, train_labels):
    if label in digits_to_save and not saved_digits[label]:
        # Convert the image to a PIL Image object
        pil_image = Image.fromarray(image)
        # Save the image as a PNG file
        pil_image.save(os.path.join(output_dir, f'digit_{label}.png'))
        # Mark this digit as saved
        saved_digits[label] = True
        print(f'Saved digit {label} as digit_{label}.png')
    # Break the loop if all digits have been saved
    if all(saved_digits.values()):
        break

print('All specified digit images have been saved.')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Saved digit 5 as digit_5.png
Saved digit 4 as digit_4.png
Saved digit 1 as digit_1.png
Saved digit 9 as digit_9.png
Saved digit 2 as digit_2.png
Saved digit 3 as digit_3.png
Saved digit 6 as digit_6.png
Saved digit 7 as digit_7.png
Saved digit 8 as digit_8.png
All specified digit images have been saved.


## Train MNIST model for digit recognition


In [25]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape and normalize the data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Filter out zeros and convert labels to 0-8 (for digits 1-9)
train_mask = (y_train > 0) & (y_train <= 9)
test_mask = (y_test > 0) & (y_test <= 9)

x_train, y_train = x_train[train_mask], y_train[train_mask] - 1
x_test, y_test = x_test[test_mask], y_test[test_mask] - 1

# Create the model
model = Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),  # Explicit input layer
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(9, activation='softmax')  # 9 outputs for digits 1-9
])

# Compile and train
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, 
                   epochs=5, 
                   validation_data=(x_test, y_test),
                   batch_size=32)

# Save the model
model.save('digit_model.h5')
print("Model saved successfully!")

Epoch 1/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8640 - loss: 0.4203 - val_accuracy: 0.9844 - val_loss: 0.0485
Epoch 2/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9754 - loss: 0.0780 - val_accuracy: 0.9904 - val_loss: 0.0299
Epoch 3/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9839 - loss: 0.0550 - val_accuracy: 0.9906 - val_loss: 0.0272
Epoch 4/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9873 - loss: 0.0412 - val_accuracy: 0.9907 - val_loss: 0.0268
Epoch 5/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9893 - loss: 0.0342 - val_accuracy: 0.9931 - val_loss: 0.0229


Model saved successfully!


## Solve the puzzle

In [35]:
import tensorflow as tf

def extract_digits(processed_image, contour):
    # Get perspective transform
    pts = contour.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    
    # Order points: top-left, top-right, bottom-right, bottom-left
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    # Get width and height of the grid
    widthA = np.sqrt(((rect[2][0] - rect[3][0]) ** 2) + ((rect[2][1] - rect[3][1]) ** 2))
    widthB = np.sqrt(((rect[1][0] - rect[0][0]) ** 2) + ((rect[1][1] - rect[0][1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    
    heightA = np.sqrt(((rect[1][0] - rect[2][0]) ** 2) + ((rect[1][1] - rect[2][1]) ** 2))
    heightB = np.sqrt(((rect[0][0] - rect[3][0]) ** 2) + ((rect[0][1] - rect[3][1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    
    # Apply perspective transform
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(processed_image, M, (maxWidth, maxHeight))
    
    # Load trained digit recognition model
    model = tf.keras.models.load_model('digit_model.h5')
    
    # Create 9x9 grid
    cell_height = maxHeight // 9
    cell_width = maxWidth // 9
    grid = np.zeros((9, 9), dtype=int)
    # Extract each cell
    for i in range(9):
        for j in range(9):
            cell = warped[i*cell_height:(i+1)*cell_height, j*cell_width:(j+1)*cell_width]
            
            # Add padding to cell
            pad = 5
            cell = cell[pad:-pad, pad:-pad]
                
            # Preprocess cell for model
            cell = cv2.resize(cell, (28, 28))
            cell = cv2.GaussianBlur(cell, (5,5), 0)
            _, cell = cv2.threshold(cell, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
            
            # Ensure digit is white on black background
            if np.mean(cell[0:5, 0:5]) > 127:
                cell = 255 - cell
                
            # Prepare input for model - match training data preprocessing
            cell = cell.reshape(1, 28, 28, 1)
            cell = cell.astype('float32') / 255.0
            
            # Get model prediction
            prediction = model.predict(cell, verbose=0)
            digit = np.argmax(prediction[0]) + 1  # Add 1 since model predicts 0-8 for digits 1-9
            
            # Only accept predictions with high confidence
            if np.max(prediction[0]) > 0.3:
                grid[i][j] = digit
            else:
                grid[i][j] = 0
                
    return grid

def solve_puzzle(image, contour):
    # First extract the grid
    grid = extract_digits(image, contour)
    print(grid)
    
    # Solve the Sudoku
    if solve_sudoku(grid):
        return grid
    else:
        return None

# Add the solving functions from previous response here
def is_valid(board, num, pos):
    # Check row
    for x in range(len(board[0])):
        if board[pos[0]][x] == num and pos[1] != x:
            return False
            
    # Check column
    for x in range(len(board)):
        if board[x][pos[1]] == num and pos[0] != x:
            return False
    
    # Check 3x3 box
    box_x = pos[1] // 3
    box_y = pos[0] // 3
    for i in range(box_y * 3, box_y * 3 + 3):
        for j in range(box_x * 3, box_x * 3 + 3):
            if board[i][j] == num and (i, j) != pos:
                return False
    
    return True

def find_empty(board):
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0:
                return (i, j)
    return None

def solve_sudoku(board):
    empty = find_empty(board)
    if not empty:
        return True
    
    row, col = empty
    
    for num in range(1, 10):
        if is_valid(board, num, (row, col)):
            board[row][col] = num
            
            if solve_sudoku(board):
                return True
            
            board[row][col] = 0
    
    return False

# Solve both puzzles
solution1 = solve_puzzle(puzzle1_processed, puzzle1_contour)
solution2 = solve_puzzle(puzzle2_processed, puzzle2_contour)

# Display results
if solution1 is not None:
    print("Solution for Puzzle 1:")
    print(np.array(solution1))
else:
    print("Could not solve Puzzle 1")

if solution2 is not None:
    print("\nSolution for Puzzle 2:")
    print(np.array(solution2))
else:
    print("Could not solve Puzzle 2")

0.4330234
0.88909674
0.73285955
0.14286034
0.14286034
0.20765854
0.99967337
0.9999988
0.14286034
0.14286034
0.28831974
0.9999589
0.88794327
0.14286034
0.14286034
0.14286034
0.549113
0.9985129
0.28761825
0.8786603
0.9999999
0.14286034
0.14286034
0.9999982
0.9767131
0.97592735
0.4912032
0.22035226
0.14286034
0.14286034
1.0
0.14286034
0.14286034
0.14286034
0.14286034
0.5766772
0.14286034
0.89610183
0.14286034
0.96141046
0.14286034
0.14286034
1.0
0.7002913
0.99573475
0.98542017
0.14286034
0.14286034
0.14286034
0.14286034
0.14286034
0.94279605
0.14286034
0.14286034
0.3068825
0.22431225
0.48631167
0.96336734
0.99999976
0.3971767
0.9951444
0.14286034
0.14286034
0.9953798
0.62803537
0.3141681
0.14286034
0.8330071
0.14286034
0.99999845
0.14286034
0.14286034
1.0
0.9876324
0.32191592


0.14286034
0.555913
0.14286034
0.5338051
0.14286034
0.9999989
[[9 4 6 0 0 0 8 3 0]
 [0 0 8 8 0 0 0 4 5]
 [0 7 3 0 0 2 1 9 8]
 [0 0 0 2 0 0 0 0 1]
 [0 9 0 1 0 0 3 3 8]
 [8 0 0 0 0 0 4 0 0]
 [8 0 1 3 2 8 9 0 0]
 [4 6 8 0 1 0 2 0 0]
 [2 8 8 0 9 0 9 0 3]]
0.28831974
0.99822456
0.9999995
0.9984438
0.36043528
0.14286034
0.14286034
0.14286034
0.83825415
0.99999917
0.28831974
0.8890118
0.6449865
0.99954164
0.14286034
0.14286034
0.9775277
0.14286034
0.28761825
0.999203
0.14286034
0.14286034
0.14286034
0.14286034
0.9999796
1.0
0.14286034
0.79506814
0.9997471
0.14286034
0.14286034
1.0
0.14286034
0.14286034
0.14286034
0.14286034
0.30255356
0.997436
0.89877474
0.14286034
0.97701204
0.14286034
0.9824891
0.836071
0.14286034
0.32790223
0.29968688
0.14286034
0.14286034
0.99802643
0.14286034
0.14286034
0.9989323
0.99999976
0.32517096
0.99999905
0.9644885
0.14286034
0.14286034
0.14286034
0.14286034
0.944473
0.14286034
0.32191592
0.93429804
0.23510462
0.20061472
0.99961036
0.96673566
0.9867628
0.14286034
